# Understanding LangGraph - A Podcast production workflow
In this notebook we are going to be building a workflow that can take a topic from a human and produce an AI generated podcast.

> Indented block



In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "playground"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

## Concepts to keep in Mind with LangGraph
- Model: Model represents your LLM configuration
- Agent: Represents your agent configuration
- Tools: Tools are functions that can be executed from arguments generated by an LLM model in our case this will be [Tavily search](https://tavily.com/), this tool has an out of the box wrapper in Langchain, we will also be creating a custom tool which is a Text to Speech tool for transcribing text to audio.
- Nodes
- Edges
- Workflow

Lets install some libraries here
- langgraph
- openai
- langchain
- langchain_openai
- tavily-python
- pydub



In [ ]:
!pip install --quiet -U langgraph langchain langchain_openai tavily-python openai pydub cohere

## Tools Setup
We need two tools here, one to search for a topic and help with research and another to convert text to audio as well as combine the audio in a light post-production

### Search Tool
We are using Tavily as search and the reason is that it is purpose built for Large language model retrieval. This tool comes pre-built with Langchain as well, so we can use it there. If you want to learn how to set it up, watch this [video](https://youtu.be/23q-B8MiDE4?si=CmziQc24YFPqGrAB)

### Text to Speech Tool
For this we are going to be using OpenAI's TTS API

### Audio Mixing Tool
For this we will be using PyDub


































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































































####Search Tool


In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=5)


#### Text to Speech Audio

In [ ]:
from langchain_core.tools import tool
from openai import OpenAI
from pathlib import Path

@tool
def text_to_speech(text: str, voice: str, output_filename: str) -> str:
    """
    Converts text to speech using OpenAI's TTS API, saving the audio file locally with a dynamic filename.

    Args:
        text (str): The text to convert to speech.
        voice (str): The voice model to use for speech synthesis. Options include:
                     - 'alloy', 'echo', 'fable', 'onyx', 'nova', 'shimmer'.
        output_filename (str): The name for the output audio file, including file extension (e.g., 'output.mp3').

    Returns:
        str: The path to the saved audio file.
    """
    client = OpenAI()
    speech_file_path = Path(output_filename)
    response = client.audio.speech.create(model="tts-1-hd", voice=voice, input=text)
    response.stream_to_file(speech_file_path)
    return str(speech_file_path)




#### Audio Mixing Tool
This tool is intended to help organize the audio segements generated and organizing them into one full podcast episode

In [ ]:
from langchain_core.tools import tool
from pydub import AudioSegment
from typing import List

@tool
def edit_podcast_audio(segments: List[str], pauses_between_segments: int = 1000, output_filename: str = "final_podcast_episode.mp3") -> str:
    """
    Edits a podcast episode by combining audio segments with specified pauses between them, ensuring consistent volume.

    Args:
        segments (List[str]): List of paths to audio segment files.
        pauses_between_segments (int): Duration of pause between segments in milliseconds. Default is 1000.
        output_filename (str): The name for the output podcast file, including file extension (e.g., 'episode.mp3').

    Returns:
        str: The path to the saved podcast episode.
    """
    podcast_episode = AudioSegment.silent(duration=0)  # Initialize an empty audio segment

    for segment_path in segments:
        segment = AudioSegment.from_file(segment_path)  # Load the segment
        podcast_episode += AudioSegment.silent(duration=pauses_between_segments) + segment  # Append with pause

    podcast_episode = podcast_episode.normalize()  # Normalize volume
    podcast_episode.export(output_filename, format='mp3')  # Export the edited podcast

    return output_filename


#### Setup the tools
Now we have defined our tools, the next step is to set the fools for exection

In [ ]:
# Define the tools we want to use
tools = [
    tavily_tool,  # Built-in search tool via Tavily
    text_to_speech,  # Our custom text to speech tool
    edit_podcast_audio # Audio Mix Tool
]


In [ ]:
from langgraph.prebuilt import ToolExecutor
tool_executor = ToolExecutor(tools)

#### Setting up the model
We are going to be using OpenAI Chat Models, for this example we will be using GPT4 Turbo

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model = "gpt-4-0125-preview", temperature=0, streaming=True)

#### Create Agents Function
Here we get to create our Agent fuction which would be neeeded to create Agents

In [ ]:
import json

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    ChatMessage,
    FunctionMessage,
    HumanMessage,
)
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph
from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation


def create_agent(llm, tools, system_message: str):
    """Create an agent."""
    functions = [format_tool_to_openai_function(t) for t in tools]

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards answering the question."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    return prompt | llm.bind_functions(functions)

#### Define State
For this example we would be working with Message States - A list of all messages as we go from state to state

In [ ]:
import operator
from typing import Annotated, List, Sequence, Tuple, TypedDict, Union

from langchain.agents import create_openai_functions_agent
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from typing_extensions import TypedDict


# This defines the object that is passed between each node
# in the graph. We will create different nodes for each agent and tool
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str

#### Agent Nodes
Agent Nodes representing functionality performed by each Agent

In [ ]:
import functools

def agent_node(state, agent, name):
  result = agent.invoke(state)
  if isinstance(result, FunctionMessage):
    pass
  else:
    result = HumanMessage(**result.dict(exclude={"type", "name"}), name=name)
  return {
      "messages": [result],
      "sender": name
  }

#Podcast Planner Agent and Node
podcast_planner_agent = create_agent(
    model,
    [tavily_tool, text_to_speech],
    system_message="""
You are tasked with creating a structured script for a podcast episode. The script should consist of a series of interactions between the host and the guest based on the provided topic and information from the research.

For each part of the dialogue, clearly specify whether it's the host speaking or the guest. Also, assign a suitable voice model for text-to-speech conversion for each segment. Use the following voice models based on the character:

- Host segments: Use the 'alloy' voice model.
- Guest segments: Use the 'fable' voice model.

The output should be a list where each item is a dictionary with keys 'speaker', 'text', and 'voice', indicating the speaker (host or guest), their line of dialogue, and the voice model to use.

Example output format:
[
    {"speaker": "host", "text": "Welcome to our podcast, where we explore the latest in technology.", "voice": "alloy"},
    {"speaker": "guest", "text": "Thank you for having me, it's great to be here to share my experiences.", "voice": "fable"},
    {"speaker": "host", "text": "Can you tell us about your current project?", "voice": "alloy"},
    {"speaker": "guest", "text": "Certainly! I've been working on a new AI platform that...", "voice": "fable"},
    ...
]

Your task is to generate a similar structured script, ensuring each dialogue segment between the host and guest is well-defined and allocates the appropriate voice model for the text-to-speech conversion process.
"""

)

podcast_planner_node = functools.partial(agent_node, agent=podcast_planner_agent, name="Podcast Plannner")

#Research Agent

research_agent = create_agent(
    model,
    [tavily_tool],
    system_message="You should provide accurate data for both the Podcast Planner to use"

)
research_node = functools.partial(agent_node, agent=research_agent, name="Researcher")

#Editor Agent
editor_agent = create_agent(
    model,
    [tavily_tool],
    system_message="""
You are the Editor, tasked with a critical review of the podcast script before it goes to audio production. Your review must focus on three key areas:

1. Flow and Clarity: Ensure that the dialogue between the host and guest flows naturally and is clear for listeners. The script must be optimized for text-to-speech conversion, paying close attention to pronunciation, pacing, and expression.

2. File System Uniqueness: Verify that the filenames suggested for each audio segment are unique and follow a logical naming convention. This is crucial to avoid overwriting files and to ensure seamless integration in the final podcast episode.

3. Content Quality and Rewrites: Assess the content for its informational value, engagement, and suitability for the podcast's audience. You have the authority to rewrite parts of the dialogue to enhance clarity, engagement, or factual accuracy. Your goal is to refine the script to a point where it translates effectively into an engaging audio experience.

After your review, you may either approve the script for audio production or make necessary adjustments. If adjustments are made, clearly indicate the changes and provide updated filenames if necessary. Your input will directly influence the quality of the final podcast episode.
"""
)
editor_node = functools.partial(agent_node, agent=editor_agent, name="Editor")


#Audio Mixer Agent
audio_agent = create_agent(
    model,
    [text_to_speech, edit_podcast_audio],
    system_message="""
You are responsible for producing the final audio for the podcast episode. Take the structured script provided by the Podcast Planner, which contains segments marked with 'speaker' (either 'host' or 'guest'), the 'text' for each segment, and the 'voice' model to use.

For each segment, use the 'text_to_speech' tool to generate audio, specifying the 'text' and 'voice' as provided. Ensure each segment is saved as a separate audio file.

After generating all segments, use the 'edit_podcast_audio' tool to combine these audio files into one seamless podcast episode. The audio files should be combined in the order they are provided in the script, with appropriate pauses between segments to simulate a natural conversation flow.

Your output should be the path to the final combined podcast episode audio file.
"""

)
audio_node = functools.partial(agent_node, agent=audio_agent, name="Audio")


#### Define Tool Node
Tool Node represents a node that executes tools using the Tool innovocation function

In [ ]:
def tool_node(state):
    """This runs tools in the graph

    It takes in an agent action and calls that tool and returns the result."""
    messages = state["messages"]
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    tool_input = json.loads(
        last_message.additional_kwargs["function_call"]["arguments"]
    )
    # We can pass single-arg inputs by value
    if len(tool_input) == 1 and "__arg1" in tool_input:
        tool_input = next(iter(tool_input.values()))
    tool_name = last_message.additional_kwargs["function_call"]["name"]
    action = ToolInvocation(
        tool=tool_name,
        tool_input=tool_input,
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(
        content=f"{tool_name} response: {str(response)}", name=action.tool
    )
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

#### Define Edge Logic
The Edge logic is intened to provide the handling of the flow depending on specific conditions. If function arguments are returned then the next action in the flow would be to call the tool node, if there is a FINAL ANSWER text, simply meeans we need to end the flow and if those are not the case we need to progress the flow to the next node.

In [ ]:
def router(state):
    # This is the router
    messages = state["messages"]
    last_message = messages[-1]
    if "function_call" in last_message.additional_kwargs:
        # The previus agent is invoking a tool
        return "call_tool"
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return "end"
    return "continue"


#### Define the Graph

In [ ]:
from langgraph.graph import END, StateGraph

# Assuming AgentState, tool_node, and all other nodes are defined elsewhere in your code.

# Initialize the graph with the state type
workflow = StateGraph(AgentState)

# Add nodes for each part of the workflow
workflow.add_node("Researcher", research_node)
workflow.add_node("Podcast Planner", podcast_planner_node)
workflow.add_node("Editor", editor_node)  # Editor node added
workflow.add_node("Audio", audio_node)

# Node to handle tool invocations
workflow.add_node("call_tool", tool_node)

# Define the flow from Researcher to Podcast Planner
workflow.add_conditional_edges(
    "Researcher",
    router,
    {"continue": "Podcast Planner", "call_tool": "call_tool", "end": END},
)

# Define the flow from Podcast Planner to Editor
workflow.add_conditional_edges(
    "Podcast Planner",
    router,
    {"continue": "Editor", "call_tool": "call_tool", "end": END},
)

# Define the flow from Editor to Audio
workflow.add_conditional_edges(
    "Editor",
    router,
    {"continue": "Audio", "call_tool": "call_tool", "end": END},
)

# Define the flow for the Audio node
workflow.add_conditional_edges(
    "Audio",
    router,
    {"continue": END, "call_tool": "call_tool", "end": END},  # After Audio, the process ends or calls a tool
)

# Define how the graph transitions back from calling a tool
workflow.add_conditional_edges(
    "call_tool",
    lambda state: state["sender"],  # Routes back to the original agent who invoked the tool
    {
        "Researcher": "Researcher",
        "Podcast Planner": "Podcast Planner",
        "Editor": "Editor",
        "Audio": "Audio",
    },
)

# Set the entry point for the graph to "Researcher"
workflow.set_entry_point("Researcher")

# Compile the graph
graph = workflow.compile()


In [ ]:
from langchain_core.messages import HumanMessage

# Start the graph with an initial message that represents the podcast topic or question
initial_message = HumanMessage(
    content="Research the upcoming 2024 superbowl, some sub plots about Pat Mahomes, Kelce and Taylor swift as well as well as the halftime performance"
)

# Stream through the graph, processing each step according to the defined workflow
for state in graph.stream(
    {"messages": [initial_message], "sender": "User"},  # Initial state with the message from the user
    {"recursion_limit": 150}  # Set a limit to prevent infinite loops
):
    print(state)  # Print out the state at each step to observe the progress
    print("----")

# This loop will go through the research, planning, audio production, and potentially tool invocation steps,
# depending on how your nodes and router logic are set up.




{'Researcher': {'messages': [HumanMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"2024 Super Bowl date and location"}', 'name': 'tavily_search_results_json'}}, name='Researcher')], 'sender': 'Researcher'}}
----
{'call_tool': {'messages': [FunctionMessage(content='tavily_search_results_json response: [{\'url\': \'https://sports.yahoo.com/2024-super-bowl-time-date-022915688.html?fr=sycsrp_catchall\', \'content\': \'2024 Super Bowl: Time, date, location for 49ers-Chiefs big game  2024 Super Bowl: Date, time, location and how to watch When: Sunday, Feb. 11, 2024 Kickoff: 6:30 p.m. ET  This article originally appeared on USA TODAY: When is the Super Bowl? Time, date, and location for big game  There are just a few days left until the 2024 Super Bowl between the Kansas City Chiefs and the San Francisco 49ers.Feb 7, 2024; Las Vegas, NV, USA; A general overall view of San Francisco 49ers and Kansas City Chiefs logos on the Allegiant Stadium facade. Mandatory Cr

<ipython-input-31-b41387b9a182>:22: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


{'call_tool': {'messages': [FunctionMessage(content='text_to_speech response: SB2024_Intro_Host.mp3', name='text_to_speech')]}}
----
{'Audio': {'messages': [HumanMessage(content='', additional_kwargs={'function_call': {'arguments': '{"text":"Thanks for having me. It\'s an electrifying time for football fans everywhere, especially with the Super Bowl heading to Las Vegas for the first time. The matchup between the Chiefs and the 49ers is one for the ages.","voice":"fable","output_filename":"SB2024_Guest_Response1.mp3"}', 'name': 'text_to_speech'}}, name='Audio')], 'sender': 'Audio'}}
----


<ipython-input-31-b41387b9a182>:22: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


{'call_tool': {'messages': [FunctionMessage(content='text_to_speech response: SB2024_Guest_Response1.mp3', name='text_to_speech')]}}
----
{'Audio': {'messages': [HumanMessage(content='', additional_kwargs={'function_call': {'arguments': '{"text":"Welcome to the Gridiron Spotlight, your premier podcast for all things NFL. Today, we\'re gearing up for the 2024 Super Bowl LVIII, a historic showdown taking place in Las Vegas for the first time. The Kansas City Chiefs and the San Francisco 49ers are set to battle it out in what promises to be an epic clash. I\'m your host, and joining me today is our NFL analyst, who\'s here to break down the game\'s biggest storylines. Great to have you with us.","voice":"alloy","output_filename":"SB2024_Intro_Host_Updated.mp3"}', 'name': 'text_to_speech'}}, name='Audio')], 'sender': 'Audio'}}
----


...

{'call_tool': {'messages': [FunctionMessage(content='text_to_speech response: SB2024_Host_Usher_Performance.mp3', name='text_to_speech')]}}
----
{'Audio': {'messages': [HumanMessage(content='', additional_kwargs={'function_call': {'arguments': '{"text":"Just that this Super Bowl is a testament to the talent and hard work of everyone involved, from the players to the coaches, and even the performers. It\'s going to be a great game, and I can\'t wait to see how it all unfolds.","voice":"fable","output_filename":"SB2024_Guest_Final_Thoughts.mp3"}', 'name': 'text_to_speech'}}, name='Audio')], 'sender': 'Audio'}}
----


<ipython-input-31-b41387b9a182>:22: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


{'call_tool': {'messages': [FunctionMessage(content='text_to_speech response: SB2024_Guest_Final_Thoughts.mp3', name='text_to_speech')]}}
----
{'Audio': {'messages': [HumanMessage(content='', additional_kwargs={'function_call': {'arguments': '{"text":"Thank you for joining us and sharing your insights. And thank you, listeners, for tuning in. Don\'t miss the 2024 Super Bowl, and remember to keep the Gridiron Spotlight on for all your NFL news and analysis. Until next time.","voice":"alloy","output_filename":"SB2024_Host_Closing.mp3"}', 'name': 'text_to_speech'}}, name='Audio')], 'sender': 'Audio'}}
----


<ipython-input-31-b41387b9a182>:22: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


{'call_tool': {'messages': [FunctionMessage(content='text_to_speech response: SB2024_Host_Closing.mp3', name='text_to_speech')]}}
----
{'Audio': {'messages': [HumanMessage(content='', additional_kwargs={'function_call': {'arguments': '{"segments":["SB2024_Intro_Host_Updated.mp3","SB2024_Guest_LasVegas.mp3","SB2024_Host_Question_Mahomes.mp3","SB2024_Guest_Mahomes_Analysis.mp3","SB2024_Host_Kelce_Question.mp3","SB2024_Guest_Kelce_Analysis.mp3","SB2024_Host_Usher_Performance.mp3","SB2024_Guest_Usher_Performance.mp3","SB2024_Guest_Final_Thoughts.mp3","SB2024_Host_Closing.mp3"]}', 'name': 'edit_podcast_audio'}}, name='Audio')], 'sender': 'Audio'}}
----
{'call_tool': {'messages': [FunctionMessage(content='edit_podcast_audio response: final_podcast_episode.mp3', name='edit_podcast_audio')]}}
----
{'Audio': {'messages': [HumanMessage(content='FINAL ANSWER: The final combined podcast episode audio file has been successfully created. You can access it here: [final_podcast_episode.mp3](sandbox:/f